## Code AST construction and preprocessing

对于IR，我们使用最普通的retrieval方法，因为CSN的数据量尚允许；注意code search和IR（其子集是semantic search）数据类型有所不同

IBA类似MNR loss；数据集使用CodeSearchNet（AdvTest）；数据类型不同且模型不同时，fusion layer就很重要了

使用sentence-transformer；由于训练数据必须是text并且必须使用bi-encoder，因此只作为encoder部分（暂定text encoder不训练否则可能学习率不兼容）

最主要比较的是GNN模型的选择，以及和baseline的比较

## evaluate和ModelContra还是在当黑箱用

In [ ]:
# Install torch geometric
!pip install torch-scatter -f https://pytorch-geometric.com/whl/torch-1.11.0+cu113.html
!pip install torch-sparse -f https://pytorch-geometric.com/whl/torch-1.11.0+cu113.html
!pip install torch-geometric
!pip install transformers
!pip install -U sentence-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://pytorch-geometric.com/whl/torch-1.11.0+cu113.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.6/107.6 kB 11.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for torch-scatter: filename=torch_scatter-2.1.1-cp39-cp39-linux_x86_64.whl size=3518810 sha256=7a1fe66a856977236b8be995f41d4dd5fde0b8443df4396ebc7a513230034c17
  Stored in directory: /root/.cache/pip/wheels/d5/0c/18/11b4cf31446c5d460543b0fff930fcac3a3f8a785e5c73fb15
Successfully built torch-scatter
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://pytorch-geometric.com/whl/torch-1.11.0+cu113.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.2/209.2 kB 16.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for torch-sparse: filename=torch_sparse-0.6.17-cp39-cp39-linux_x86_64.whl 

In [ ]:
# https://docs.python.org/3/library/ast.html#ast-helpers
# some of the classes may vary according to the version of Python, e.g. "Match" is not present before 3.10

class_list = [
  "AST",
  "Constant", "FormattedValue", "JoinedStr",
  "List", "Tuple", "Set", "Dict", "Name",
  "Load", "Store", "Del", "Starred", "Expr",
  "UnaryOp", "UAdd", "USub", "Not", "Invert",
  "BinOp", "Add", "Sub", "Mult", "Div", "FloorDiv",
  "Mod", "Pow", "LShift", "RShift", "BitOr",
  "BitXor", "BitAnd", "MatMult", "BoolOp",
  "And", "Or", "Compare", "Eq", "NotEq",
  "Lt", "LtE", "Gt", "GtE", "Is", "IsNot",
  "In", "NotIn", "Call", "keyword", "IfExp",
  "Attribute", "NamedExpr", "Subscript", "Slice",
  "ListComp", "SetComp", "GeneratorExp", "DictComp",
  "comprehension", "Assign", "AnnAssign", "AugAssign",
  "Raise", "Assert", "Delete", "Pass", "Import",
  "ImportFrom", "alias", "If", "For", "While",
  "Break", "Continue", "Try", "TryStar", "ExceptHandler",
  "With", "withitem", "Match", "match_case",
  "MatchValue", "MatchSingleton", "MatchSequence",
  "MatchStar", "MatchMapping", "MatchClass",
  "MatchAs", "MatchOr", "FunctionDef", "Lambda",
  "arguments", "arg", "Return", "Yield", "YieldFrom",
  "Global", "Nonlocal", "ClassDef", "AsyncFunctionDef",
  "Await", "AsyncFor", "AsyncWith"
]

class_dict = {class_list[i]: i for i in range(len(class_list))}

In [ ]:
# based on https://gist.github.com/joshmarlow/4001898
# AST到NetworkX的转化，NetworkX到PyG图的转化，以及节点特征的构建
import ast
import networkx as nx
from sklearn.feature_extraction.text import CountVectorizer

class viz_walker(ast.NodeVisitor):
  def __init__(self):
    self.stack = []
    self.graph = nx.Graph()
    self.vectorizer = CountVectorizer(lowercase=False, vocabulary=class_dict, binary=True)

  def generic_visit(self, stmt):
    node_name = str(stmt)

    parent_name = None

    if self.stack:
      parent_name = self.stack[-1]

    self.stack.append(node_name)

    self.graph.add_node(node_name, x=self.vectorizer.transform([stmt.__class__.__name__]).toarray().squeeze())

    if parent_name:
      self.graph.add_edge(node_name, parent_name)

    super(self.__class__, self).generic_visit(stmt)

    self.stack.pop()

## GNN model as code encoder

In [ ]:
import torch
import torch.nn as nn
import pandas as pd
import torch.nn.functional as F
print(torch.__version__)

import os

from torch_geometric.nn import GCNConv, global_mean_pool
from torch_geometric.utils import from_networkx

2.0.0+cu118


In [ ]:
# based on codebase of CS224W
class GCN(nn.Module): # 之后拓展到更多模型
  def __init__(self, input_dim, hidden_dim, output_dim, num_layers, dropout, return_embeds=False):
    super(GCN, self).__init__()

    # A list of GCNConv layers
    self.convs = nn.ModuleList()

    # A list of 1D batch normalization layers
    self.bns = nn.ModuleList()

    # The log softmax layer
    self.softmax = nn.LogSoftmax(dim=-1)

    self.convs.append(GCNConv(input_dim, hidden_dim))
    for _ in range(num_layers - 2):
      self.convs.append(GCNConv(hidden_dim, hidden_dim))
    self.convs.append(GCNConv(hidden_dim, output_dim))

    for _ in range(num_layers - 1):
      self.bns.append(nn.BatchNorm1d(hidden_dim))

    self.relu = nn.ReLU()

    # Probability of an element getting zeroed
    self.dropout = nn.Dropout(p=dropout)

    # Skip classification layer and return node embeddings
    self.return_embeds = return_embeds

  def reset_parameters(self):
    for conv in self.convs:
      conv.reset_parameters()
    for bn in self.bns:
      bn.reset_parameters()

  def forward(self, x, adj_t):
    for i in range(len(self.convs) - 1):
      x = self.convs[i](x, adj_t) # 可以是edge list也可以是adjacency matrix
      x = self.bns[i](x)
      x = self.relu(x)
      x = self.dropout(x)

    out = self.convs[-1](x, adj_t) # 如果输出embedding，那么维度是output_dim
    if self.return_embeds == False:
      out = self.softmax(out)

    return out

In [ ]:
### GCN to predict graph property
# based on codebase of CS224W
class GCN_Graph(nn.Module):
  def __init__(self, input_dim, hidden_dim, num_layers, dropout):
    super(GCN_Graph, self).__init__()

    # Node embedding model
    self.gnn_node = GCN(input_dim, hidden_dim,
      hidden_dim, num_layers, dropout, return_embeds=True)

    self.pool = global_mean_pool # 从节点embedding出发，用mean pooling得到graph embedding

  def reset_parameters(self):
    self.gnn_node.reset_parameters()
    self.linear.reset_parameters()

  def forward(self, batched_data):
    x, edge_index, batch = batched_data.x, batched_data.edge_index, batched_data.batch

    out = self.gnn_node(x, edge_index)
    out = self.pool(out, batch)

    return out

## The whole retrieval model

In [ ]:
class ModelContra(nn.Module):
  def __init__(self, text_encoder, code_encoder):
    super(ModelContra, self).__init__()
    self.text_encoder = text_encoder
    self.code_encoder = code_encoder
    self.mlp = nn.Sequential(nn.Linear(768*4, 768),
      nn.Tanh(),
      nn.Linear(768, 1),
      nn.Sigmoid())
    self.loss_func = nn.BCELoss()

  def forward(self, code_inputs, nl_inputs, labels, return_vec=False):
    # nl_inputs是text的list，code_inputs是PyG Data的list
    nl_vec = self.text_encoder.encode(nl_inputs, convert_to_tensor=True)
    code_vec = self.code_encoder(code_inputs)
    if return_vec:
      return code_vec, nl_vec

    nl_vec = nl_vec.unsqueeze(1).repeat([1, 64, 1])
    code_vec = code_vec.unsqueeze(0).repeat([64, 1, 1])
    logits = self.mlp(torch.cat((nl_vec, code_vec, nl_vec-code_vec, nl_vec*code_vec), 2)).squeeze(2) # (Batch, Batch)
    matrix_labels = torch.diag(labels).float()  # (Batch, Batch)
    poss = logits[matrix_labels==1]
    negs = logits[matrix_labels==0]

    # loss = self.loss_func(logits, matrix_labels)
    # bce equals to -(torch.log(1-logits[matrix_labels==0]).sum() + torch.log(logits[matrix_labels==1]).sum()) / (bs*bs)
    loss = - (torch.log(1 - negs).mean() + torch.log(poss).mean())
    predictions = (logits.gather(0, torch.arange(64, device=loss.device).unsqueeze(0)).squeeze(0) > 0.5).int()
    return loss, predictions

## Data and model preparation

In [ ]:
import random

import numpy as np
from torch.utils.data import Dataset

from torch_geometric.loader import DataLoader

import json

from tqdm import tqdm, trange

from transformers import AdamW, get_linear_schedule_with_warmup
from sentence_transformers import SentenceTransformer

In [ ]:
class InputFeatures(object):
  def __init__(self, code_data, query, label, idx):
    # Here code_data is a PyG Data ovject
    self:code_data = code_data
    self.query = query
    self.label = label
    self.idx = idx


def convert_examples_to_features(js):
  label = js['label']

  code = js['code']
  parsed = ast.parse(code)
  nw = viz_walker()
  nw.visit(parsed)
  code_data = from_networkx(nw.graph)

  nl = js['doc']

  return InputFeatures(code_data, nl, label, js['idx'])


class TextDataset(Dataset):
  def __init__(self, data_path=None):
    # file：json文件，每一个dict中包括：idx, query, doc, code(或者叫function_tokens，list形式), docstring_tokens(list形式)
    self.examples = []
    self.data = []
    with open(data_path, 'r') as f:
      self.data = json.load(f)
    for js in self.data:
      self.examples.append(convert_examples_to_features(js))

  def __len__(self):
    return len(self.examples)

  def __getitem__(self, i):
    return self.examples[i].code, self.examples[i].query, self.examples[i].label


class RetrievalDataset(Dataset):
  def __init__(self, retrieval_code_base, data_path=None):
    self.codes = []
    self.data = []
    self.examples = []  # codebase用code和code当成6k pair; testdata用query和code当成pair来算; 分别是examples的0-6267和6267-6767
    code_file = retrieval_code_base
    data_file = data_path

    with open(code_file, 'r') as f:
      self.codes = json.loads(f.read())
    for code, code_id in self.codes.items():
      js = {'code': code, 'doc': code, 'label': 1, 'idx': code_id}
      self.examples.append(convert_examples_to_features(js))

    with open(data_file, 'r') as f:
      self.data = json.load(f)
    for js in self.data:
      new_js = {'code': js['code'], 'doc': js['doc'], 'label': js['label'], 'idx': js['retrieval_idx']}
      self.examples.append(convert_examples_to_features(new_js))

  def __len__(self):
    return len(self.examples)

  def __getitem__(self, i):
    return self.examples[i].code, self.examples[i].query, self.examples[i].label


def set_seed(seed=45):
  random.seed(seed)
  os.environ['PYHTONHASHSEED'] = str(seed)
  np.random.seed(seed)
  torch.manual_seed(seed)
  torch.cuda.manual_seed(seed)
  torch.backends.cudnn.deterministic = True

In [ ]:
def evaluate(model):
    print("Running evaluation...")
    model.eval()
    all_first_vec = []
    all_second_vec = []
    for batch in eval_dataloader:
        code_inputs = batch[0].to(device)
        nl_inputs = batch[1]
        labels = batch[2].to(device)
        with torch.no_grad():
            code_vec, nl_vec = model(code_inputs, nl_inputs, labels, return_vec=True)
            all_first_vec.append(code_vec.cpu())
            all_second_vec.append(nl_vec.cpu())
    code_vectors = torch.cat(all_first_vec, 0).squeeze()[:6267, :]
    query_vectors = torch.cat(all_second_vec, 0).squeeze()[6267:, :]
    assert code_vectors.size(0) == 6267
    assert query_vectors.size(0) == 500

    repeat_nl_vec = query_vectors.unsqueeze(1).to(device)
    scores = []
    with torch.no_grad():
        for idx in trange(6267):
            repeat_code_vec = code_vectors[idx, :].unsqueeze(0).repeat([500, 1, 1]).to(device)
            logits = model.mlp(torch.cat((repeat_nl_vec, repeat_code_vec, repeat_nl_vec - repeat_code_vec, repeat_nl_vec * repeat_code_vec),2)).squeeze(2)
            scores.append(logits.cpu())

    scores = torch.cat(scores, 1)
    results = []
    mrr = 0
    for idx in range(len(scores)):
        rank = torch.argsort(-scores[idx]).tolist()
        example = eval_dataset.examples[idx+6267]
        item = {}
        item['ans'] = example.idx
        item['rank'] = rank
        item['rr'] = 1 / (rank.index(example.idx)+1)
        mrr += item['rr']
        results.append(item)
    mrr = mrr / len(scores)
    return mrr

In [ ]:
"""
sentence-transformer的保存问题
需要去掉不能被parse的code
"""
device = torch.cuda.current_device() if torch.cuda.is_available() else 'cpu'
print(f"using {device}.")

train_data_path = "./cosqa-retrieval-train-19604-qra-switch-28624.json"
eval_data_path = "./cosqa-retrieval-dev-500.json"
retrieval_code_base = "./code_idx_map.txt"
output_dir = "./outputs"

# set_seed(45)

text_encoder = SentenceTransformer("all-mpnet-base-v2", device=device)
code_encoder = GCN_Graph(input_dim=len(class_dict), hidden_dim=768, num_layers=3, dropout=0.5)
model = ModelContra(text_encoder, code_encoder)
model.to(device)

for param in text_encoder.parameters(): # we first test with fixed text encoder
  param.requires_grad = False

train_dataset = TextDataset(train_data_path)
train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=64)
eval_dataset = RetrievalDataset(retrieval_code_base, eval_data_path)
eval_dataloader = DataLoader(eval_dataset, shuffle=False, batch_size=64)

num_train_epochs = 5
save_steps = 500

optimizer = AdamW(model.parameters(), lr=0.01)
scheduler = get_linear_schedule_with_warmup(optimizer, len(train_dataloader), len(train_dataloader) * num_train_epochs)

step = 0
best_mrr = 0.0

for idx in range(num_train_epochs):
  for batch in tqdm(train_dataloader):
    code_inputs = batch[0].to(device)
    nl_inputs = batch[1]
    labels = batch[2].to(device)

    model.train()
    optimizer.zero_grad()
    loss, predictions = model(code_inputs, nl_inputs, labels)
    loss.backward()
    optimizer.step()
    scheduler.step()
    step += 1

    if step % save_steps == 0:
      mrr = evaluate(model)
      print(f"Mrr = {mrr}")
      if mrr >= best_mrr:
        best_mrr = mrr
        print(f"Save checkpoint!")

        checkpoint_prefix = 'checkpoint-best-mrr'
        output_dir = os.path.join(output_dir, checkpoint_prefix)
        if not os.path.exists(output_dir):
          os.makedirs(output_dir)
        model_to_save = model.module if hasattr(model, 'module') else model

        torch.save(model_to_save.state_dict(), os.path.join(output_dir, 'pytorch_model.bin'))

using 0.


SyntaxError: ignored

In [ ]:
device = torch.cuda.current_device() if torch.cuda.is_available() else 'cpu'
print(f"using {device}.")
 
test_data_file = "./cosqa-retrieval-test-500.json"
retrieval_code_base = "./code_idx_map.txt"
retrieval_predictions_output = "./outputs/retrieval_outputs.txt"
output_dir = "./outputs"

# set_seed(45)

text_encoder = SentenceTransformer("all-mpnet-base-v2", device=device)
code_encoder = GCN_Graph(input_dim=len(class_dict), hidden_dim=768, num_layers=3, dropout=0.5)
model = ModelContra(text_encoder, code_encoder)
model.load_state_dict(torch.load(os.path.join(output_dir, 'checkpoint-best-mrr/pytorch_model.bin')))
model.to(device)

test_dataset = RetrievalDataset(retrieval_code_base, test_data_file)
test_dataloader = DataLoader(test_dataset, shuffle=False, batch_size=64)

print("Running Test...")
model.eval()
all_first_vec = []
all_second_vec = []
for batch in test_dataloader:
    code_inputs = batch[0].to(device)
    nl_inputs = batch[1]
    labels = batch[2].to(device)
    with torch.no_grad():
        code_vec, nl_vec = model(code_inputs, nl_inputs, labels, return_vec=True)
        all_first_vec.append(code_vec.cpu())
        all_second_vec.append(nl_vec.cpu())
code_vectors = torch.cat(all_first_vec, 0).squeeze()[:6267, :]
query_vectors = torch.cat(all_second_vec, 0).squeeze()[6267:, :]
assert code_vectors.size(0) == 6267
assert query_vectors.size(0) == 500

repeat_nl_vec = query_vectors.unsqueeze(1).to(device)
scores = []
with torch.no_grad():
    for idx in trange(6267):
        repeat_code_vec = code_vectors[idx, :].unsqueeze(0).repeat([500, 1, 1]).to(device)
        logits = model.mlp(torch.cat((repeat_nl_vec, repeat_code_vec, repeat_nl_vec - repeat_code_vec, repeat_nl_vec * repeat_code_vec),2)).squeeze(2)
        scores.append(logits.cpu())

scores = torch.cat(scores, 1)
results = []
mrr = 0
for idx in range(len(scores)):
    rank = torch.argsort(-scores[idx]).tolist()
    example = test_dataset.examples[idx+6267]
    item = {}
    item['ans'] = example.idx
    item['rank'] = rank
    item['rr'] = 1 / (rank.index(example.idx)+1)
    mrr += item['rr']
    results.append(item)
mrr = mrr / len(scores)
print(f"Final test MRR: {mrr}")
with open(retrieval_predictions_output, 'w') as f:
    json.dump(results, f, indent=4)